In [ ]:
! pip install datasets transformers
!apt install git-lfs # Is an extension to Git that helps you to store large files

In [26]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


In [30]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_from_scratch_notebook", framework="pytorch")

# Train a Language Model
Language model learns the structure of language by predicting words

1. Casual Language Modeling(CLM) -
The goal is to predict next token in a sequence.The model only sees past tokens[we use the casual mask] so it does not cheat by seeing future words.
2. Masked Language Modeling (MLM)-
The goal is to predict missing tokens that are masked in the input
ex- The dog [MASK] the cat



# Preparing Dataset

In [7]:
from datasets import load_dataset
datasets=load_dataset('wikitext','wikitext-2-raw-v1')

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [12]:
# It will display randomly chosen words in table format.
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(datasets["train"])

,text
0,
1,"[ A ] n economy which emphasizes consumer 's goods is characterized by communication industries largely dependent on advertising and by constant efforts to reach the largest number of readers or listeners ; an economy emphasizing producer 's goods is characterized by communications industries largely dependent on government support . As a result of this contrast , a common public opinion in Russia and the West is hard to achieve . \n"
2,
3,
4,
5,"RIDOT also has long @-@ range plans to construct direct freeway connections linking Route 4 north with I @-@ 95 south and I @-@ 95 north with Route 4 south . As of November 2010 , environmental studies are being prepared for a reconfiguration of the interchange . \n"
6,
7,= Alice in Chains = \n
8,
9,


# Casual Language Modeling
For casual language modeling we are going to take all text in our dataset and concatenate them after they are tokenized.Then we will split them in certain sequence length.

In [14]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

Will tokenize all our texts with the same vocabulary by using `AutoTokenizer`

In [15]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(tokenizer_checkpoint)

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/396k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [9]:
# Will define a function
def tokenize_function(examples):
  return tokenizer(examples['text'])

In [16]:
# batched=True -> Tells the map() function to pass a batch of examples.
# num_proc=4 -> This runs the mapping in parallel using 4 processes.
# remove_columns=['text'] -> It removes the original column from dataset.
tokenized_datasets=datasets.map(tokenize_function,batched=True,num_proc=4,remove_columns=['text'])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [17]:
# Text has been replaced by input_ids in the dataset.
tokenized_datasets['train'][1]

{'input_ids': [238, 8576, 9441, 2987, 238, 252],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

We will concatenate all the text and then split into small chunks of certain `block_size`

In [18]:
block_size=128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [47]:
lm_datasets=tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [48]:
#  now the samples contain chunks of block_size contiguous tokens
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large'

In [22]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [49]:
from transformers import Trainer, TrainingArguments

In [50]:
training_args = TrainingArguments(
    f"{model_checkpoint}-wikitext2",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [52]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,6.495800,6.469968
2,6.127100,6.186572


Epoch,Training Loss,Validation Loss
1,6.495800,6.469968
2,6.127100,6.186572
3,5.947800,6.100558


TrainOutput(global_step=6747, training_loss=6.318465487699394, metrics={'train_runtime': 2340.6946, 'train_samples_per_second': 23.059, 'train_steps_per_second': 2.882, 'total_flos': 3525678710784000.0, 'train_loss': 6.318465487699394, 'epoch': 3.0})

In [ ]:
# Evaluate our model
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()

In [ ]:
# Fetch our own model
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("OmBhandwalkar/my-awesome-model")

# Masked language modeling
For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by [MASK]) and the labels will be adjusted to only include the masked tokens. If you use a tokenizer you trained yourself, make sure the [MASK] token is among the special tokens you passed during training!

In [33]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"
from transformers import AutoTokenizer,AutoModelForMaskedLM,TrainingArguments,Trainer
from datasets import load_dataset

In [10]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [19]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [22]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [29]:
training_args = TrainingArguments(
    "test-clm",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_checkpoint}-wikitext2",
)

In [31]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15,mlm=False)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")